# Running an Individual Test
This notebook shows how to run individual metrics or thresholds tests that is part of the ValidMind Developer Framework. These instructions include the code required to:

- Load a demo dataset
- Prepocess the raw dataset
- Train a model for testing
- Set up test inputs and run the test
- Initialize the required ValidMind objects
- Run the test
- Log the test results to ValidMind

## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

In [ ]:
%pip install -q validmind

## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook. As you will see later, documentation projects are useful because they act as containers for model documentation and validation reports and they enable you to organize all of your documentation work in one place. 

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Customer Churn Model`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [ ]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

In [ ]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

## Load the demo dataset

In [ ]:
from validmind.datasets.classification import customer_churn as demo_dataset

# You can also import customer_churn like this:
# from validmind.datasets.classification import taiwan_credit as demo_dataset

df = demo_dataset.load_data()

### Prepocess the raw dataset

In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)

## Train a model for testing

We train a simple customer churn model for our test.

In [ ]:
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

## Set up test inputs and run the test

### Initialize ValidMind objects

In [ ]:
vm_train_ds = vm.init_dataset(
    input_id="train_dataset",
    dataset=train_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    input_id="test_dataset",
    dataset=test_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_model = vm.init_model(
    model,
    input_id="model"
)

### Assign predictions to the datasets
We can now use the `assign_predictions()` method from the Dataset object to link existing predictions to any model. If no prediction values are passed, the method will compute predictions automatically:

In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
    prediction_values=list(model.predict(x_train))
)
vm_test_ds.assign_predictions(
    model=vm_model,
    prediction_values=list(model.predict(x_val))
)

### Run the test

Individual tests can be easily run by calling the `run_test` function provided by the `validmind.tests` module. The function takes the following arguments:

- `test_id`: The ID of the test to run. To find a particular test and get its, refer to the `explore_tests.ipynb` notebook
- `params`: A dictionary of parameters for the test. These will override any `default_params` set in the test definition. Refer to the `explore_tests.ipynb` notebook to find the default parameters for a test.

You can then pass in any inputs for the test as keyword arguments. Most likely, these will be `dataset` and `model` objects. Again, you may refer to the `explore_tests.ipynb` notebook to find the required inputs for a test.

In [ ]:
test = vm.tests.run_test(
    test_id="validmind.model_validation.sklearn.TrainingTestDegradation",
    params={}, # can be used to set overrides to the test's default parameters
    inputs = {
        "model":vm_model,
        "datasets": (vm_train_ds, vm_test_ds)
    }
)

### Log the test results to ValidMind

After the test has been run, you can save the results to ValidMind by calling the `log` method of the test object returned after running the test:

In [ ]:
test.log()